In [25]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [26]:
result_path = '/root/projects/prompt_tuning_with_constraint_ICLR/results'

In [27]:
os.listdir(result_path)

['bert-base-uncased',
 'roberta-large',
 'stat.ipynb',
 'robertabase.tar',
 'FacebookAI',
 'roberta-base']

In [4]:
df = pd.read_csv(f'/root/projects/prompt_tuning_with_constraint_ICLR/results/bert-base-uncased/SST-2_0SST-2-gamma-5e-07-lr-0.001-lr_LM-0.001-epoch-1-num_of_init_text-1-seed-50similarityNone-layer--3.csv')
df.head()

,epoch,eval_loss,eval_accuracy,layer,prompt
0,1.0,0.295038,0.878440,-2,SST-2_0
1,1.0,0.296718,0.871560,-1,SST-2_0
2,1.0,0.297138,0.877294,0,SST-2_0
3,1.0,0.299428,0.869266,1,SST-2_0
4,1.0,0.303188,0.866972,2,SST-2_0


In [6]:
df['layer'][0]

-2

In [32]:
def read_csv(file_name):
    df = pd.read_csv(file_name)
    return df, len(df)
    
def read_files(model_name, task_name):
    file_names = []
    all_files = os.listdir(f'{result_path}/{model_name}')
    layer_num = 0
    file_count = 0
    all_layer_dict = {}

    for file in all_files:
        filename = f'{result_path}/{model_name}/{file}'
        if task_name not in file:
            continue
        df, layer_num = read_csv(filename)
        file_count += 1
        for i in range(0, layer_num):
            
            delta = df['eval_accuracy'][i] - 0#- df['eval_accuracy'][0]
            if not all_layer_dict.get(df['layer'][i]):
                all_layer_dict[df['layer'][i]] = delta
            else:
                all_layer_dict[df['layer'][i]] += delta
    
    for k, v in all_layer_dict.items():
        all_layer_dict[k] = v/file_count

    return all_layer_dict, file_count


In [33]:
read_files('bert-base-uncased', 'SST-2')

({-2: 0.8658256880733944,
  -1: 0.8599006116207951,
  0: 0.7236238532110092,
  1: 0.7219036697247706,
  2: 0.724006116207951,
  3: 0.7232415902140672,
  4: 0.7217125382262997,
  5: 0.7232415902140672,
  6: 0.7238149847094801,
  7: 0.7236238532110092,
  8: 0.7220948012232417,
  9: 0.7226681957186546,
  10: 0.7217125382262997,
  11: 0.7217125382262997},
 6)

In [35]:
read_files('bert-base-uncased', 'sst-5')

({-2: 0.3364214350590372,
  -1: 0.38801089918256126,
  0: 0.3852861035422343,
  1: 0.3883742052679382,
  2: 0.38455949137148043,
  3: 0.38510445049954584,
  4: 0.38764759309718433,
  5: 0.37874659400544963,
  6: 0.38383287920072656,
  7: 0.39037238873751134,
  8: 0.3840145322434151,
  9: 0.3865576748410536,
  10: 0.3847411444141689,
  11: 0.3912806539509536},
 5)

In [37]:
read_files('bert-base-uncased', 'subj')

({-2: 0.9273999999999999,
  -1: 0.9386000000000001,
  0: 0.9349999999999999,
  1: 0.9342,
  2: 0.9347999999999999,
  3: 0.9328,
  4: 0.9327,
  5: 0.9338000000000001,
  6: 0.9336,
  7: 0.9333,
  8: 0.9333000000000002,
  9: 0.9334,
  10: 0.9339000000000001,
  11: 0.9345000000000001},
 5)

In [24]:
read_files('bert-base-uncased', 'trec')

({-1: 0.20400000000000001,
  0: 0.04999999999999999,
  1: 0.04999999999999999,
  2: 0.04999999999999999,
  3: 0.04999999999999999,
  4: 0.04999999999999999,
  5: 0.04999999999999999,
  6: 0.04999999999999999,
  7: 0.04999999999999999,
  8: 0.04999999999999999,
  9: 0.04999999999999999,
  10: 0.04999999999999999,
  11: 0.04999999999999999},
 5)

In [38]:
read_files('roberta-base', 'SST-2')

({-2: 0.9025229357798166,
  -1: 0.9013761467889908,
  0: 0.9051987767584097,
  1: 0.9032874617737002,
  2: 0.9044342507645259,
  3: 0.9021406727828746,
  4: 0.9021406727828745,
  5: 0.8990825688073395,
  6: 0.9044342507645259,
  7: 0.9025229357798165,
  8: 0.8994648318042814,
  9: 0.8998470948012232,
  10: 0.8971712538226301,
  11: 0.8990825688073394},
 3)

In [39]:
read_files('roberta-base', 'sst-5')

({-2: 0.30093854072055704,
  -1: 0.3151680290644868,
  0: 0.31819557977596125,
  1: 0.3415077202543142,
  2: 0.33424159854677565,
  3: 0.332425068119891,
  4: 0.31940660006055105,
  5: 0.3184983348471087,
  6: 0.32485619134120497,
  7: 0.32122313048743567,
  8: 0.41568271268543744,
  9: 0.4211323039660914,
  10: 0.41114138661822586,
  11: 0.42960944595821976},
 3)

In [40]:
read_files('roberta-base', 'subj')

({-2: 0.9188000000000001,
  -1: 0.9212,
  0: 0.9177,
  1: 0.9185000000000001,
  2: 0.9184999999999999,
  3: 0.9183,
  4: 0.9182,
  5: 0.9181000000000001,
  6: 0.9185000000000001,
  7: 0.9187,
  8: 0.9196,
  9: 0.9170999999999999,
  10: 0.9187,
  11: 0.9169},
 5)

In [41]:
read_files('roberta-base', 'trec')

({-2: 0.45599999999999996,
  -1: 0.34440000000000004,
  0: 0.44480000000000003,
  1: 0.43079999999999996,
  2: 0.3936,
  3: 0.442,
  4: 0.4532,
  5: 0.4436,
  6: 0.40199999999999997,
  7: 0.3708,
  8: 0.45280000000000004,
  9: 0.41479999999999995,
  10: 0.4076000000000001,
  11: 0.40040000000000003},
 5)